In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [2]:
mario_path_files = !ls TheVGLC/Super\ Mario\ Bros/Paths/*txt
levels = []
vocab = set([';','{','}'])
for file in mario_path_files:
    with open(file) as input_file:
        level = []
        for row in input_file:
            level.append(list(row.rstrip()))
            vocab |= set(row.rstrip())
        levels.append(level)
print(vocab)

v2i = {v:i for i,v in enumerate(sorted(vocab))}
i2v = {i:v for v,i in v2i.items()}

t_levels = []
for level in levels:
    t_level = []
    for row in range(len(level[0])):
        column = []
        for col in range(len(level)):
            column.append(level[col][row])
        t_level.append(column)
    t_levels.append(t_level)
            
chunks = []
chunk_size = 32
for level in t_levels:
    for x in range(len(level)-chunk_size):
        chunk = level[x:x+chunk_size]
        chunk = [''.join(c) for c in chunk]
        chunks.append(';'.join(chunk)+'}')


        

{'x', '[', '}', 'X', 'Q', 'S', '?', 'b', ']', '<', 'E', 'o', 'B', '-', ';', '>', '{'}


In [239]:
class RNNVAE(nn.Module):
    def __init__(self,e_layers,d_layers,
                 dropout,vocab_size,
                 enc_size,
                 dec_size,latent_size,
                 rnn_type):
        super(RNNVAE, self).__init__()
        self.latent_dim = latent_size
        self.enc_size = enc_size
        self.dec_size = dec_size
        self.embed = nn.Embedding(vocab_size,enc_size)
        self.encoder = rnn_type(enc_size,hidden_size=enc_size,
                              num_layers=e_layers,batch_first=True,
                                dropout=dropout, bidirectional=True)
        
        self.to_mu = nn.Linear(enc_size*2,latent_size)
        self.to_logvar = nn.Linear(enc_size*2,latent_size)
        self.latent_to_h = nn.Linear(latent_size,dec_size*d_layers)
        self.latent_to_c = nn.Linear(latent_size,dec_size*d_layers)
        
        self.decoder = rnn_type(enc_size,hidden_size=dec_size,
                              num_layers=d_layers,dropout=dropout,
                                batch_first=True,)
        self.hidden_to_vocab = nn.Linear(dec_size,vocab_size)
        self.CE = nn.CrossEntropyLoss()
    def forward(self,input_sequence,start_tok):
        
        batch_size, seq_length = input_sequence.size(0), input_sequence.size(1)
        device = input_sequence.device
        embedded = self.embed(input_sequence)
        out , _= self.encoder(embedded)
        
        f_enc = out[:,0,:out.size()[-1]//2]
        r_enc = out[:,-1,out.size()[-1]//2:]
        enc = torch.cat((f_enc,r_enc),dim=-1)
        mu,logvar = self.to_mu(enc), self.to_logvar(enc)
        
        std = torch.exp(0.5 * logvar)
        z = torch.randn([batch_size,self.latent_dim],device=device)
        z = z * std + mu        
        dec_h = self.latent_to_h(z)
        dec_h = dec_h.view(batch_size,-1, self.dec_size).permute(1,0,2).contiguous()
        dec_c = self.latent_to_c(z)
        dec_c = dec_c.view(batch_size,-1, self.dec_size).permute(1,0,2).contiguous()

        dec_inp = torch.cat((
             self.embed(torch.ones(batch_size,1,dtype=torch.long,device=device)*start_tok),
            embedded[:,:-1,:]),dim=1)        
        
        dec_out, _ =self.decoder(dec_inp,(dec_h,dec_c))
        dec_logit = self.hidden_to_vocab(dec_out)
        
        loss = self.CE(dec_logit.view(batch_size*seq_length,-1),input_sequence.view(-1))
        KL_div =  -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return loss, KL_div, dec_logit
    
# class RNNVAE(nn.Module):
#     def __init__(self,e_layers,d_layers,
#                  dropout,vocab_size,
#                  enc_size,
#                  dec_size,latent_size,
#                  rnn_type):
#         super(RNNVAE, self).__init__()
#         self.latent_dim = latent_size
#         self.enc_size = enc_size
#         self.dec_size = dec_size
#         self.embed = nn.Embedding(vocab_size,enc_size)
#         self.dec_embed = nn.Embedding(vocab_size,enc_size)
#         self.encoder = rnn_type(enc_size,hidden_size=enc_size,
#                               num_layers=e_layers,batch_first=True,
#                                 dropout=dropout, bidirectional=True)
        
#         self.to_mu = nn.Linear(enc_size*2,latent_size)
#         self.to_logvar = nn.Linear(enc_size*2,latent_size)
#         self.latent_to_d = nn.Linear(latent_size,enc_size)
        
#         self.decoder = rnn_type(enc_size,hidden_size=dec_size,
#                               num_layers=d_layers,dropout=dropout,
#                                 batch_first=True,)
#         self.hidden_to_vocab = nn.Linear(dec_size,vocab_size)
#         self.CE = nn.CrossEntropyLoss()
#     def forward(self,input_sequence,start_tok):
        
#         batch_size, seq_length = input_sequence.size(0), input_sequence.size(1)
#         device = input_sequence.device
#         embedded = self.embed(input_sequence)
#         out , _= self.encoder(embedded)
        
#         f_enc = out[:,0,:out.size()[-1]//2]
#         r_enc = out[:,-1,out.size()[-1]//2:]
#         enc = torch.cat((f_enc,r_enc),dim=-1)
#         mu,logvar = self.to_mu(enc), self.to_logvar(enc)
        
#         std = torch.exp(0.5 * logvar)
#         z = torch.randn([batch_size,self.latent_dim],device=device)
#         z = z * std + mu        
#         dec_h = self.latent_to_d(z)
#         dec_h = dec_h.unsqueeze(-1).expand((batch_size,self.enc_size,seq_length)).permute(0,2,1)
        
#         dec_embedded = self.dec_embed(input_sequence)
#         dec_inp = torch.cat((
#             self.dec_embed (torch.ones(batch_size,1,dtype=torch.long,device=device)*start_tok),
#             dec_embedded[:,:-1,:]),dim=1) + dec_h 
        
#         dec_out, _ =self.decoder(dec_inp)
#         dec_logit = torch.tanh(self.hidden_to_vocab(dec_out))
        
#         loss = self.CE(dec_logit.view(batch_size*seq_length,-1),input_sequence.view(-1))
#         KL_div =  -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
#         return loss, KL_div, dec_logit
    
#a = torch.tensor([[1,2],[2,3],[3,4]])
#b = torch.zeros(3,7,2)
#print(a,b)
#print(a.unsqueeze(-1).expand((3,2,7)).permute(0,-1,1))

#BEST SO FAR: rnn_vae = RNNVAE(2,1,0.3,len(vocab),512,128,128,nn.LSTM) 
#BEST SO FAR: rnn_vae = RNNVAE(2,1,0.3,len(vocab),512,128,64,nn.LSTM) 
rnn_vae = RNNVAE(4,2,0.4,len(vocab),512,128,128,nn.LSTM)   
optimizer = optim.Adam(rnn_vae.parameters(),lr=1e-3)

In [263]:
from tqdm import tqdm_notebook
import random
import numpy as np
device = 'cuda'
epochs = 100
batch_size = 32
show_every = 16
rnn_vae.to(device)
warmup = 98
for _ in range(10):
    losses = []
    
    annealing_rate = 1
    annealing = annealing_rate
    for epoch in tqdm_notebook(range(epochs)):
        random.shuffle(chunks)
        for batch in tqdm_notebook(range(0,len(chunks),batch_size)):
            rnn_vae.train()
            rnn_vae.zero_grad()
            batch = chunks[batch:batch+batch_size]
            batch = [[v2i[t] for t in c] for c in batch]
            input_sequence =torch.tensor(batch).to(device)
            loss, KL_div, dec_logit = rnn_vae(input_sequence,v2i['{'])
            loss_KL_div = loss + KL_div*(1.0-annealing)
            losses.append((loss_KL_div.item(),loss.item(),KL_div.item()))
            loss_KL_div.backward()
            torch.nn.utils.clip_grad_norm_(rnn_vae.parameters(), 1)
            optimizer.step()
            if len(losses) % show_every == 0:
                losses_ = np.array(losses)
                print(epoch,np.mean(losses_[-show_every*2:,0]),
                      np.mean(losses_[-show_every*2:,1]),
                      np.mean(losses_[-show_every*2:,2]))
                
        if epoch >= warmup:
            annealing *= annealing_rate

0 0.02192357589956373 0.02192357589956373 0.04550126567482948
0 0.0223123092437163 0.0223123092437163 0.20175848994404078
0 0.02299940591910854 0.02299940591910854 0.7382310228422284
0 0.023402490187436342 0.023402490187436342 1.8258758317679167



1 0.02263636770658195 0.02263636770658195 3.852382503449917
1 0.021795182838104665 0.021795182838104665 6.715537503361702
1 0.021813949046190828 0.021813949046190828 9.467474892735481
1 0.021825937263201922 0.021825937263201922 11.78766542673111
1 0.02177251852117479 0.02177251852117479 13.912623316049576



2 0.021566760377027094 0.021566760377027094 15.52105975151062
2 0.021409267093986273 0.021409267093986273 17.222414821386337
2 0.021543522889260203 0.021543522889260203 19.241355180740356
2 0.021481378586031497 0.021481378586031497 20.868059873580933
2 0.02139008231461048 0.02139008231461048 22.39669442176819



3 0.021451984415762126 0.021451984415762126 23.476331919431686
3 0.02118468820117414 0.02118468820117414 25.080781251192093
3 0.021117884200066328 0.021117884200066328 27.269959628582
3 0.02160801540594548 0.02160801540594548 29.03676664829254
3 0.022428255062550306 0.022428255062550306 30.81424778699875



4 0.02228120295330882 0.02228120295330882 32.716631561517715
4 0.021553426864556968 0.021553426864556968 36.798328667879105
4 0.02170836104778573 0.02170836104778573 41.768447279930115
4 0.02215149934636429 0.02215149934636429 45.74900233745575
4 0.021993761823978275 0.021993761823978275 50.0510938167572



...

57 0.009163026392343454 0.009163026392343454 23859.240509033203
57 0.009075294889044017 0.009075294889044017 23841.834442138672
57 0.008101503946818411 0.008101503946818411 24321.901000976562
57 0.007615237380377948 0.007615237380377948 24349.42120361328
57 0.00759230756375473 0.00759230756375473 24354.825622558594



58 0.007564333820482716 0.007564333820482716 23800.287841796875
58 0.0076386290602386 0.0076386290602386 23785.864868164062
58 0.00782015077129472 0.00782015077129472 24285.343627929688
58 0.008110538736218587 0.008110538736218587 24378.022094726562



59 0.007976732144015841 0.007976732144015841 23980.648315429688
59 0.007388769547105767 0.007388769547105767 23987.568298339844
59 0.007219944076496176 0.007219944076496176 24391.508728027344
59 0.007366679550614208 0.007366679550614208 24367.474243164062
59 0.007499331928556785 0.007499331928556785 24384.656616210938



60 0.007582582024042495 0.007582582024042495 23852.17950439453


KeyboardInterrupt: 

In [266]:
import torch.nn.functional as F
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (batch size x vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0
        # scatter sorted tensors to original indexing
        indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)
        
        logits[indices_to_remove] = filter_value
    return logits

def autoencode(self,input_sequence,start_tok,temp):
        
    batch_size, seq_length = input_sequence.size(0), input_sequence.size(1)
    device = input_sequence.device
    embedded = self.embed(input_sequence)
    out , _= self.encoder(embedded)

    f_enc = out[:,0,:out.size()[-1]//2]
    r_enc = out[:,-1,out.size()[-1]//2:]
    enc = torch.cat((f_enc,r_enc),dim=-1)
    mu,logvar = self.to_mu(enc), self.to_logvar(enc)

    std = torch.exp(0.5 * logvar)
    z = torch.randn([batch_size,self.latent_dim],device=device)
    z = z * std + mu        
    dec_h = self.latent_to_h(z)
    dec_h = dec_h.view(batch_size,-1, self.dec_size).permute(1,0,2).contiguous()
    dec_c = self.latent_to_c(z)
    dec_c = dec_c.view(batch_size,-1, self.dec_size).permute(1,0,2).contiguous()

    dec_inp = self.embed(torch.ones(batch_size,1,dtype=torch.long,device=device)*start_tok)
    tokens = []    
    scores = []
    for _ in range(seq_length):
        dec_out, _ =self.decoder(dec_inp,(dec_h,dec_c))
        dec_logit = self.hidden_to_vocab(dec_out[:,-1,:]).view(batch_size,-1)
        dec_logit = top_k_top_p_filtering(dec_logit,top_k=5)
        next_token = torch.multinomial(F.softmax(dec_logit/temp, dim=-1), num_samples=1)
        
        tokens.append(next_token.detach())
        chosen_scores = []
        for ii,tok in enumerate(tokens[-1]):
            chosen_scores.append(dec_logit[ii,tok].item())
        scores.append(chosen_scores)
        dec_inp =torch.cat((dec_inp,self.embed(next_token)
            ),dim=1)
        

    return tokens,scores
def prettify(level):
    levelstr = ''
    level = level.split('}')[0].split(';')
    level = [c for c in level if len(c) == len(level[0])]
    width = len(level)
    height = len(level[0]) 
    for y in range(height):
        for x in range(width):

            levelstr += level[x][y]
        levelstr += '\n'
    return levelstr
import numpy as np
samples = 10
best_scores = np.ones(input_sequence.size()[0])*-np.inf
best_samples = [None]*input_sequence.size()[0]
for _ in range(samples):
    with torch.no_grad():
        rnn_vae.eval()
        encoded, scores = autoencode(rnn_vae,input_sequence,v2i['{'],1.2)
    scores = np.array(scores)
    scores = np.mean(scores,axis=0)
    for ii,(sc,be) in enumerate(zip(scores,best_scores)):
        if sc > be:
            best_scores[ii] = sc
            generation = []
            for batch in encoded:
                token = batch[ii].item()
                generation.append(token)
            best_samples[ii] = prettify(''.join([i2v[t] for t in generation]))
    print(best_scores)

[15.6856197  16.08251897 16.39017353 15.55805208 16.9875865  15.12934466
 16.26177156 16.18846285 16.60630578 15.49253858 16.42261915 17.57366859
 15.50084398 16.58568913 15.47446743 14.82117793 14.90068846 15.74464795
 15.84105402 15.47875916 15.69369439 16.81253864 15.3741402  16.61268112
 14.97086222 14.71170027 15.75621819 15.38820898 16.89626328 14.76126389
 15.3680995  15.93959753]
[16.00975214 16.08251897 16.39017353 15.73464356 16.99148258 15.33457346
 16.26177156 16.18846285 16.60630578 16.14044306 16.42261915 17.57366859
 15.50084398 16.58568913 15.94358279 14.9287245  14.90068846 16.73871683
 15.8413534  15.99081571 15.69369439 16.8788575  15.3741402  16.61268112
 15.97219607 15.8162407  16.26489526 15.38820898 16.89626328 14.81177598
 15.3680995  16.11560906]
[16.00975214 16.08356291 16.3902896  16.67416088 16.99148258 15.63870339
 16.86594318 16.18846285 16.88381394 16.87784178 16.42261915 17.57366859
 16.20335845 16.58568913 15.94358279 14.9287245  14.90068846 16.73871683

In [267]:
for b,g in enumerate(best_samples):
    print('original')
    print(prettify(''.join([i2v[t.item()] for t in input_sequence[b,:]])))
    print('generated')
    print(g)

original
--------------------------------
--------------------------------
--------------------------------
--------------------------------
---oo---------------------------
--------------------------------
--------------------------------
--------------------------------
x-------------------------------
-x------------------------------
--x-----xxxx--------------------
---x---xx<>-x------------------x
-E-Exxxx-[]-Exxxxxxxxxxxxxxxxxxx
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

generated
--------------------------------
--------------------------------
--------------------------------
--------------------------------
--------------------------------
---Q----------------------------
--------------------------------
--------------------------------
x-------------------oo----------
-x-?---------------o--o---------
--x-----------------------------
---x-----xxx--------------------
----xxxxxx--xxxxxxxxxxxxxxxxxxxx
XXXXXXXXXX--XXXXXXXXXXXXXXXXXXXX

original
--------------------------------
-------------

--------------------------xxxxxx
xxxxxxxxxxxxxxxxxxxxxxxxxxx-XXX-
SSSSSSSSSSSSSSSSSSSSSSSSSSS-----
--------------------------------
--------------------------------
--------------------------------
--------------------------------
--------------------------------
---------------E----------------
-----------E--<>----------------
----E-----<>--[]----------XX----
---<>-----[]--[]---------XXXXXX-
---[]---E-[]--[]--------XXXX----
-XXXXXXXXXXXXXXX--XXXXXXXXX-----

generated
--------------------------------
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS
--------------------------------
--------------------------------
---E----------------------------
--<>----------------------------
--[]----------------------------
--[]-------S--------------------
-S[]SS---E----E----------------<
--[]----<>---<>-------------XX-[
--[]----[]---[]------------XXX-[
E-[]----[]---[]--------E-E-XXX-[
XXXXXXXXXXXXXXXXXXXXXXXXXXXXX--X

original
--------------------------------
----------------------

In [268]:
!pwd

/home/bizon/Research/SequenceBasedLevelGeneration
